[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NNDesignDeepLearning/NNDesignDeepLearning/blob/master/10.PyTorchIntroChapter/Code/LabSolutions/PyTorchIntroLab2_Solution.ipynb)

# PyTorch Introduction Lab 2  -- Explainability

This objective of this second PyTorch lab is to experiment with explainability. You will begin by implementing the saliency method and then moving on to the integrated gradient method. For example purposes, you will begin by reloading the convolution network that you trained in the first lab, and then you will test the explainability methods on that network.

## Loading the Model

We begin by loading some useful modules. 

In [ ]:
%matplotlib inline 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np

Next we load the test dataset from MNIST.

In [ ]:
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

Before loading the model, since we only saved the model's `state_dict`, we need to create an instance of the model. We will use the same definition of `cnn_model` that we used in the previous lab. If you changed the network architecture before you saved the network, you will need to use your definition of `cnn_model`. Put the definition in the next cell.

In [ ]:
# Define the CNN model
class cnn_model(nn.Module):
    def __init__(self):
        super(cnn_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = x
        return output


Now that we have defined the cnn_model class, create an instance of that class, and load the model's state_dict using `model.load_state_dict(torch.load(PATH))`.

In [ ]:
# data_path = '/media/martin/Storage/github/DeepLearning/10.PyTorchIntroChapter/Code/data/'
model_path = '/media/martin/Storage/github/DeepLearning/10.PyTorchIntroChapter/Model/'
# model_path = '../Model/'

model = cnn_model()

model.load_state_dict(torch.load(model_path + 'cnn_model_state.pt'))

# Implementing Saliency

To implement saliency, we need to compute the gradient of the relevant network output with respect to the network input. The following method has three arguments: the network model, the input tensor to the network and the target class. It returns the gradient of the target class output with respect to the network input.

In [ ]:
def compute_input_gradient(model: nn.Module,
                           input_tensor: torch.Tensor,
                           target_class: int) -> torch.Tensor:
    """
    Compute gradient of specific class output with respect to input.

    Args:
        model: PyTorch neural network model
        input_tensor: Input tensor (should be of shape [batch_size, channels, height, width])
        target_class: Index of target class to compute gradient for

    Returns:
        Gradient tensor of same shape as input
    """
    # Set model to eval mode
    model.eval()

    # Enable gradient computation
    input_tensor.requires_grad_(True)

    # Forward pass
    output = model(input_tensor)

    # Select target class output
    target_output = output[:, target_class]

    # Compute gradient
    gradient = torch.autograd.grad(target_output.sum(),
                                   input_tensor,
                                   create_graph=True)[0]

    return gradient


Put the test data into a DataLoader so that we can feed it into the network. Use a batch size of 1, so that we can consider one image at a time.

In [ ]:
BATCH_SIZE = 1
test_loader = DataLoader(test_data, shuffle=False, batch_size=BATCH_SIZE)

Read out the image and target for a selected member of the test loader. Put this image through the model and compute the output class. Then put the model, the image and the output class as arguments to `compute_input_gradient` to compute the gradient. Multiply the gradient times the input to compute the contribution. Then display the input image and the contribution.

In [ ]:
key = 2000
for batch_idx, (data, target) in enumerate(test_loader):
    if batch_idx == key:
        output = model(data)
        break

tclass = torch.argmax(output)
print(tclass)
print(target)

gradient = compute_input_gradient(model, data, target_class=tclass)
contrib = data*gradient

plt.imshow(data[0][0].detach().numpy())
plt.show()
plt.imshow(contrib[0][0].detach().numpy())
plt.show()

# Implementing Integrated Gradient

For the integrated gradient method we can use the previous gradient calculation, but we want to compute the gradient at several points along a line between a baseline image and the current input image. Create a method that takes the model, an input image, a baseline image, the number of steps in the average, and the target class and computes the integrated gradient contribution by invoking the previous `compute_input_gradient()` method.

In [ ]:
def integrated_gradients(model: nn.Module,
                           input_tensor: torch.Tensor,
                           base_tensor: torch.Tensor,
                           n_steps: int,
                           target_class: int) -> torch.Tensor:
    """
    Compute integrated gradient contribution for a specific class output

    Args:
        model: PyTorch neural network model
        input_tensor: Input tensor (should be of shape [batch_size, channels, height, width])
        base_tensor: Base tensor (should be of shape [batch_size, channels, height, width])
        target_class: Index of target class to compute integrated gradient for

    Returns:
        Contribution tensor of same shape as input
    """

    avgradient = 0
    for alpha in np.arange(0,1,1/n_steps):
        interp = base_tensor + alpha * (input_tensor - base_tensor)
        avgradient += compute_input_gradient(model, interp, target_class=target_class)

    avgradient = avgradient/n_steps
    ig_contrib = avgradient * (input_tensor - base_tensor)
    return ig_contrib

Read out the image and target for a selected member of the test loader. Put this image through the model and compute the output class. Then put the model, the image and the output class as arguments to `integrated_gradients` to compute the integrated gradient contribution. Then display the input image and the contribution.

In [ ]:
key = 2000
for batch_idx, (data, target) in enumerate(test_loader):
    if batch_idx == key:
        output = model(data)
        break

tclass = torch.argmax(output)
print(tclass)
print(target)

base = torch.zeros(data.shape)
igcontrib = integrated_gradients(model, data, base, n_steps=10, target_class=tclass)

plt.imshow(data[0][0].detach().numpy())
plt.show()
plt.imshow(igcontrib[0][0].detach().numpy())
plt.show()

Group the data in the training, validation and testing Datasets into minibatches of size 100, using the `batch()` method.

# Using Captum

PyTorch has many built-in functions for explainability in Captum, which is an open source, extensible library for model interpretability built on PyTorch. You can find documentation [here](https://captum.ai/). We can use captum to get the same integrated gradient results. First we import `IntegratedGradients` from `captum.attr`.

In [ ]:
import captum
from captum.attr import IntegratedGradients

Next, perform the same operation as above, where you read out the image and target for a selected member of the test loader. Put this image through the model and compute the output class. Then put the model, the image and the output class as arguments to `integrated_gradients` to compute the integrated gradient contribution. Then display the input image and the contribution.

In [ ]:
# Initialize the attribution algorithm with the model
integrated_gradients_captum = IntegratedGradients(model,multiply_by_inputs=True)

key = 2000
for batch_idx, (data, target) in enumerate(test_loader):
    if batch_idx == key:
        output = model(data)
        break

tclass = torch.argmax(output)
print(tclass)
print(target)

base = torch.zeros(data.shape)
igcontrib = integrated_gradients_captum.attribute(data, baselines=base, target=tclass, n_steps=10)
plt.imshow(data[0][0].detach().numpy())
plt.show()
plt.imshow(igcontrib[0][0].detach().numpy())
plt.show()


## Explore Further

With access to the Captum library you can try different explainability methods with a minimal amount of additional code. A list of available methods can be found [here](https://captum.ai/api/index.html).

Here are some things to consider.

1. Try the occlusion method and compare with your results from saliency and integrated gradient.
1. Use the LRP method and compare.
1. After completing the TensorFlow lab 3, use captum to implement the integrated gradient method, and test it on some of the cat and dog images to find out what parts of the images are important to distinuishing cats from dogs.